In [ ]:
%%bash
nrnivmodl

In [ ]:
# Download NEURON: http://www.neuron.yale.edu/neuron/download
# Download PyNeuronToolbox: https://github.com/ahwillia/PyNeuron-Toolbox

from neuron import h
import numpy as np
import math
import pylab as plt
import scipy.linalg
from PyNeuronToolbox.record import ez_record,ez_convert
#from PyNeuronToolbox import neuromorpho,morphology
from PyNeuronToolbox.morphology import shapeplot,shapeplot_animate,load
from mpl_toolkits.mplot3d import Axes3D
from JSAnimation import IPython_display
from matplotlib import animation
from matplotlib.pyplot import cm
import warnings
warnings.filterwarnings('ignore', category=SyntaxWarning)
%matplotlib inline
np.random.seed(123456789)

import re
import pandas as pd

DC = 10 # diffusion coeff [microns^2 s^-1]

cellname = 'dgyrus'
#cellname = 'granule'
#neuromorpho.download('041015-vehicle1', filename=cellname+'_morph.swc')
#cell = morphology.load(cellname+'_morph.swc',use_axon=False)
#cell = load(cellname+'_morph.swc',use_axon=False)


In [ ]:
import sys
sys.path.append('../')
sys.path.append('../simulation/')

import sushibelt
import DG_10reg_10dv_model as model
import time
tnorm = np.sum(model.target ** 2)

In [ ]:
N=model.N

In [ ]:
tdf=pd.read_csv('../data/DG_seg_mapping.csv')
abbDG=tdf['abb']
abbT={}
segIdx={}
for i in range(N):
    abbT[abbDG[i]] = 1+ abbT.get(abbDG[i],0)
    ll=segIdx.get(abbDG[i],[])
    ll.append(i)
    segIdx[abbDG[i]] = ll


In [ ]:
from PyNeuronToolbox.morphology import dist_between,allsec_preorder

def dist_to_soma(segment):
    return dist_between(h,allsec_preorder(h)[0](0.5),segment)

#seglist in pre-order
sec_list = allsec_preorder(h)
seg_list = []
for sec in sec_list:
    locs = np.linspace(0,1,sec.nseg+2)[1:-1]
    for loc in locs:
        seg_list.append(sec(loc))
n = len(seg_list)
N = n

In [ ]:
soma = allsec_preorder(h)[0]
soma(0.5)

In [ ]:
(N,seg_list)

In [ ]:
for i in range(len(sec_list)):
    sec = sec_list[i]
    print(f"{i}: {sec.name()}, nseg={sec.nseg}, propNseg={np.round(sec.L/3.7)}, L={sec.L}, or={sec.orientation()}, hname={sec.hname()}")

In [ ]:
sec_list = allsec_preorder(h)
seg_list = []
for sec in sec_list:
    if sec.name() != 'dgyrus_morph.soma[0]':
        nseg = int(np.round(sec.L/3.7))
        locs = np.linspace(0,1,nseg+2)[1:-1]
        for loc in locs:
            seg_list.append(sec(loc))
    else:
        locs = np.linspace(0,1,3)[1:-1]
        for loc in locs:
            seg_list.append(sec(loc))
n = len(seg_list)
N = n
N

In [ ]:
(dist_to_soma(seg_list[0]),dist_to_soma(seg_list[1]),sec.L,np.linspace(0,1,sec.nseg+2),np.linspace(0,1,sec.nseg+2)[1:-1])

In [ ]:
dts = [dist_to_soma(s) for s in seg_list]
max_dts = np.max(dts) + 0.01
dts_bounds = np.linspace(0,max_dts,11)
(dts_bounds,dts_bounds[0],dts_bounds[1])

In [ ]:
p = re.compile("^([^\\[]+)\\[.+")
tps = [p.sub("\\1",seg.sec.name()) for seg in seg_list]
pd.Series(tps).drop_duplicates().tolist()

In [ ]:
seg_list = []
seg_abb = []
seg_name = []
seg_dts = [dist_to_soma(s) for s in seg_list]
for sec in sec_list:
    if sec.name() != 'dgyrus_morph.soma[0]':
        nseg = int(np.round(sec.L/3.7))
        locs = np.linspace(0,1,nseg+2)[1:-1]
        for loc in locs:
            seg_list.append(sec(loc))
            seg_name.append(sec.name())
            ldts = dist_to_soma(sec(loc))
            seg_dts.append(ldts)
            for j in range(len(dts_bounds)-1):
                print(f"ldts={ldts},j={j},dts_bounds[j]={dts_bounds[j]} and dts_bounds[j+1]={dts_bounds[j+1]}")
                if ldts >= dts_bounds[j] and ldts < dts_bounds[j+1]:
                    print(f"\trd{j+1},len(seg_abb)={len(seg_abb)}")
                    seg_abb.append(f"rd{j+1}")
                    break                    
    else:
        locs = np.linspace(0,1,3)[1:-1]
        for loc in locs:
            seg_list.append(sec(loc))
            seg_name.append(sec.name())
            ldts = dist_to_soma(sec(loc))
            seg_dts.append(ldts)
            for j in range(len(dts_bounds)-1):
                print(f"ldts={ldts},j={j},dts_bounds[j]={dts_bounds[j]} and dts_bounds[j+1]={dts_bounds[j+1]}")
                if ldts >= dts_bounds[j] and ldts < dts_bounds[j+1]:
                    print(f"\trd{j+1},len(seg_abb)={len(seg_abb)}")
                    seg_abb.append(f"rd{j+1}")
                    break
n = len(seg_list)
N = n
print([len(seg_abb),len(seg_name),len(seg_dts),len([i for i in range(N)])])

In [ ]:
(len(seg_name),len(seg_abb),len(seg_dts),len(sec_list),seg_abb)

In [ ]:
tdf=pd.read_csv('../data/DG_seg_mapping.csv')
tdf

In [ ]:
(N,tdf.shape,len(seg_list))

In [ ]:
abbDG=tdf['abb']
abbT={}
segIdx={}
for i in range(N):
    abbT[abbDG[i]] = 1+ abbT.get(abbDG[i],0)
    ll=segIdx.get(abbDG[i],[])
    ll.append(i)
    segIdx[abbDG[i]] = ll


In [ ]:
#expD=pd.read_csv('../data/DG_intensity_values_15072024.csv')
expD=pd.read_csv('../data/DG_density_values_15072024.csv')
bgSignal=1e-5
#cname='D0W3'
cname0='D0M3'
d0w = -1 * np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d0w[sidx] *= -1*expD[f"{cname0}_MEAN"][i]/len(sidx)
for i in range(N):
    if d0w[i]<0:
        d0w[i] = bgSignal
dinit = d0w/np.sum(d0w)

#cname='D7W3'
cname7='D7M3'
d7w = -1*np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d7w[sidx] *= -1 * expD[f"{cname7}_MEAN"][i]/len(sidx)
for i in range(N):
    if d7w[i]<0:
        d7w[i] = bgSignal
target = np.array(expD[f"{cname7}_MEAN"])/np.sum(expD[f"{cname0}_MEAN"]) #norm target to Day0 sum to take into accound degradation
targSD = np.array(expD[f"{cname7}_SD"])/np.sum(expD[f"{cname0}_MEAN"]) #results to fit to
tnorm = np.sum(target ** 2)
day7 = 7 * 24 * 3600 # final time point


In [ ]:
expD

In [ ]:
parnames=['F','Ctau','mProp','demand_rd1','demand_rd2','demand_rd3','demand_rd4','demand_rd5','demand_rd6',
          'demand_rd7','demand_rd8','demand_rd9','demand_rd10','dv_rd1','dv_rd2','dv_rd3','dv_rd4','dv_rd5',
          'dv_rd6','dv_rd7','dv_rd8','dv_rd9','dv_rd10']
lowb=np.array([0,-18,0,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,1.0e-07,
               1.0e-07,-18,-18,-18,-18,-18,-18,-18,-18,-18,-18,])
upbga=np.array([1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
(len(parnames),len(lowb),len(upbga),parnames[3],parnames[13])

In [ ]:
len(parnames)*21

In [ ]:
dv = np.zeros(N)
utarg = np.ones(N)
for k in range(N):
    if model.itarg[k] > 2:
        utarg[k] = lowb[model.itarg[k]]
        dv[k] = (10 ** lowb[model.itarg[k]+10])
(dv,utarg)

In [ ]:
(model.lowb[13],model.upbga[13])

In [ ]:
#par = [0.8855702481960549,-17.6142146256606,0.12385432691434908,0.9642855946085199,0.5389253732509337,
#       0.272527645830006,0.9129387693392707,0.1297446952024372,0.6107997917511535,1e-07,0.9890938472884612,
#       0.24488136124301582,0.014607108124724532,-10.154176543644859,-11.575055132626284,-6.057936626299734,
#       -6.053467113877346,-6.0477171882104965,-6.028631654679405,-6.036945094549408,-5.999154504640609,
#       -6.00544599147668,-6.0530817424270875]
#cost = 0.0027400475307714973
#chi2=1184.7252031040346
#par = [0.8001576414106837,-8.83311346686456,0.9568695929329514,0.0020138669780257502,0.025377383988998158,0.029948781242701755,
#       0.9999999999999751,0.08896057016913292,0.0004002113170755411,0.014562289941884115,1.0523901851122988e-07,
#       1.0846600291496056e-07,0.9598262098965911,-6.028288531411434]
#cost = 2.283246533053875e-05
#chi2 = 8.532544224511733
#par = [0.0001347573108234385,-7.134175640871103,0.16358340428275822,0.9535886836265206,0.9392082944358957,0.588591654675706,
#       0.9999999999998962,0.761066612409874,0.012928172225454351,0.3308015862956901,1.742438941388552e-07,
#       1.397577721676662e-07,0.9832677483391569,-6.028282727752732]
#cost = 2.283438654048628e-05
#chi2 = 8.534658143069766
#par = [0.0,-17.641758850550943,0.9899980591588442,0.24988968936615671,1.0062994989967052e-07,1e-07,0.06556921170168392,1.0016863619144738e-07,1.0000000000227276e-07,0.00458126556189493,0.0001394007972908521,1e-07,0.999999999999629,-6.063004369056134,-6.070974912176686,-6.057936625641295,-6.053467103903037,-6.04771719102088,-6.0286316572292815,-6.036945095832321,-5.99915450387148,-6.005445989961027,-6.053081737997973]
#cost = 7.174089042842454e-32
#chi2 = 1.9756083405305166e-26
par =     [0.38045811693782183,-17.999999996173983,0.9829776349705306,0.8327241060440842,0.6817423197421232,1e-07,0.001178352727833331,0.001981887509234548,0.0005868862578329533,0.9999999999872324,0.0227100824137138,0.0012922112255617517,0.6751030453233307,-5.324402793151426,-5.404319648247094,-5.413636141175232,-5.456588887637485,-5.501488091825545,-5.5319379474387596,-5.586166243439893,-5.602787801258852,-5.61233822499455,-5.629706487668917]
opt_par = [0.38045811693782183,-17.999999996173983,0.9829776349705306,0.8327241060440842,0.6817423197421232,1e-07,0.001178352727833331,0.001981887509234548,0.0005868862578329533,0.9999999999872324,0.0227100824137138,0.0012922112255617517,0.6751030453233307,-5.324402793151426,-5.404319648247094,-5.413636141175232,-5.456588887637485,-5.501488091825545,-5.5319379474387596,-5.586166243439893,-5.602787801258852,-5.61233822499455,-5.629706487668917]
cost = 1.8904206110375962e-32
chi2 = 2.144032235335936e-26
mProp = par[2]
utrace = model.calcUtrace(par)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], model.segIdx, model.expD['Abbreviation'], fun=np.sum)


In [ ]:
{'F' : par[0],
'Ctau' : par[1],
'mProp' : par[2],
'(1-mProp)' : (1-par[2]),
'demand_rd1' : par[3],
'demand_rd2' : par[4],
'demand_rd3' : par[5],
'demand_rd4' : par[6],
'demand_rd5' : par[7],
'demand_rd6' : par[8],
'demand_rd7' : par[9],
'demand_rd8' : par[10],
'demand_rd9' : par[11],
'demand_rd10' : par[12],
#'dv' : par[13],
'dv_rd1' : 10 ** par[13],
'dv_rd2' : 10 ** par[14],
'dv_rd3' : 10 ** par[15],
'dv_rd4' : 10 ** par[16],
'dv_rd5' : 10 ** par[17],
'dv_rd6' : 10 ** par[18],
'dv_rd7' : 10 ** par[19],
'dv_rd8' : 10 ** par[20],
'dv_rd9' : 10 ** par[21],
'dv_rd10' : 10 ** par[22],
'norm_factor' : (1-mProp),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - model.target) ** 2),
'err_norm' : np.sum((resF/(1-mProp) - model.target) ** 2)/tnorm,
'chi2' : np.sum(((resF/(1-mProp) - model.target)/model.targSD) ** 2),
'cost' : cost,
'Chi2' : chi2
}

In [ ]:
(n,len(dv),len(utarg))

In [ ]:
F = par[0]
Ctau = 10 ** par[1]
mProp = par[2]
dv = np.zeros(n)
utarg = bgSignal*np.ones(n)
for k in range(n):
    if model.itarg[k] > 2:
        dv[k] = (10 ** par[model.itarg[k]+10])
        utarg[k] = par[model.itarg[k]]
    else:
        dv[k] = np.nan
        utarg[k] = np.nan
#utarg /= np.sum(utarg)
bfit, afit = np.polyfit(utarg, np.array(dv), deg=1)
xseq = np.linspace(min(utarg), max(utarg), num=100)
#plt.scatter(utarg,dv)
fig, ax = plt.subplots(figsize = (9, 9))
ax.scatter(utarg, dv)
ax.plot(xseq, afit + bfit * xseq, color="k", lw=2.5)
pos = ax.get_position()
#ax.text(pos.x0, pos.y0 + pos.height, f'LR equation: $Y = {round(afit*1e6,3)}'+'\\ 10^{-6}'+f' + {round(bfit*1e6,3)}'+'\\ 10^{-6}\\ X$', fontsize=10)
ax.text(0.2,1.85e-6, f'LR equation: $Y = {round(afit*1e6,3)}'+'\\ 10^{-6}'+f' + {round(bfit*1e6,3)}'+'\\ 10^{-6}\\ X$', fontsize=10)
plt.xlabel('Demand value')
plt.ylabel('Degradation rate')
plt.show()
#plt.savefig('DG_experiment_density_10dv_degradation_fit.png')
#plt.savefig('DG_experiment_density_10dv_degradation_fit.pdf')
#plt.savefig('DG_experiment_density_10dv_degradation_fit.svg')


In [ ]:
x=np.array(range(model.expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
ax.plot(x,target,label='Target')
ax.plot(x,resM,label='mobile DC')
ax.plot(x,resF,label='detached DC')
#ax.plot(x,resF/(1-mProp),label='norm detached DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_soma.png')
#plt.savefig('bestFitCA1_20reg_1dv_soma.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_soma.svg')

In [ ]:
x=np.array(range(model.expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
ax.plot(x,target,label='Target')
#ax.plot(x,resM,label='mobile DC')
#ax.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='norm detached DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFit_DG_10reg_1dv_F08.png')
#plt.savefig('bestFit_DG_10reg_1dv_F08.pdf')
#plt.savefig('bestFit_DG_10reg_1dv_F08.svg')

In [ ]:
x=np.array(range(model.expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
ax.plot(x,expD[f"{cname0}_MEAN"],label=cname0)
#ax.plot(x,expD[f"{cname7}_MEAN"],label=cname7)
#ax.plot(x,resM,label='mobile DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_soma.png')
#plt.savefig('bestFitCA1_20reg_1dv_soma.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_soma.svg')

In [ ]:
#plt.plot(x,target,label=cname7)
plt.errorbar(x,target,targSD,label=cname7, fmt='-o')
#plt.plot(x,resF,label='detached DC')
plt.plot(x,resF/(1-mProp),label='detached norm DC')
plt.ylabel('Value')
plt.xlabel('Region number')
# show a legend on the plot
plt.legend()
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('expDG_10reg_10dv_3W_target_error.png')
#plt.savefig('expDG_10reg_10dv_3W_target_error.pdf')
#plt.savefig('expDG_10reg_10dv_3W_target_error.svg')

In [ ]:
x=np.array(range(model.expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
#ax.plot(x,expD[f"{cname0}_MEAN"],label=cname0)
ax.plot(x,expD[f"{cname7}_MEAN"],label=cname7)
#ax.plot(x,resM,label='mobile DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_20reg_1dv_soma.png')
#plt.savefig('bestFitCA1_20reg_1dv_soma.pdf')
#plt.savefig('bestFitCA1_20reg_1dv_soma.svg')

# Three weeks data

In [ ]:
expD=pd.read_csv('../data/DG_density_values_15072024.csv')
bgSignal=1e-5

cname0='D0W3'
d0w = -1 * np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d0w[sidx] *= -1*expD[f"{cname0}_MEAN"][i]/len(sidx)
for i in range(N):
    if d0w[i]<0:
        d0w[i] = bgSignal
dinit = d0w/np.sum(d0w)

cname7='D7W3'
d7w = -1*np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d7w[sidx] *= -1 * expD[f"{cname7}_MEAN"][i]/len(sidx)
for i in range(N):
    if d7w[i]<0:
        d7w[i] = bgSignal
target = np.array(expD[f"{cname7}_MEAN"])/np.sum(expD[f"{cname0}_MEAN"]) #norm target to Day0 sum to take into accound degradation
targSD = np.array(expD[f"{cname7}_SD"])/np.sum(expD[f"{cname0}_MEAN"]) #results to fit to
tnorm = np.sum(target ** 2)
day7 = 7 * 24 * 3600 # final time point


In [ ]:
model.dinit=dinit
utrace = model.calcUtrace(par)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], model.segIdx, model.expD['Abbreviation'], fun=np.sum)

In [ ]:
{'F' : par[0],
'Ctau' : par[1],
'mProp' : par[2],
'(1-mProp)' : (1-par[2]),
'demand_rd1' : par[3],
'demand_rd2' : par[4],
'demand_rd3' : par[5],
'demand_rd4' : par[6],
'demand_rd5' : par[7],
'demand_rd6' : par[8],
'demand_rd7' : par[9],
'demand_rd8' : par[10],
'demand_rd9' : par[11],
'demand_rd10' : par[12],
#'dv' : par[13],
'dv_rd1' : par[13],
'dv_rd2' : par[14],
'dv_rd3' : par[15],
'dv_rd4' : par[16],
'dv_rd5' : par[17],
'dv_rd6' : par[18],
'dv_rd7' : par[19],
'dv_rd8' : par[20],
'dv_rd9' : par[21],
'dv_rd10' : par[22],
'norm_factor' : (1-mProp),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - target) ** 2),
'err_norm' : np.sum((resF/(1-mProp) - target) ** 2)/tnorm,
'chi2' : np.sum(((resF/(1-mProp) - target)/targSD) ** 2),
'cost' : cost,
'Chi2' : chi2
}

In [ ]:
x=np.array(range(model.expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
ax.plot(x,target,label='Target')
#ax.plot(x,resM,label='mobile DC')
ax.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='norm detached DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('expDG_10reg_10dv_3W_density_target.png')
#plt.savefig('expDG_10reg_10dv_3W_density_target.pdf')
#plt.savefig('expDG_10reg_10dv_3W_density_target.svg')

In [ ]:
#plt.plot(x,target,label=cname7)
plt.errorbar(x,target,targSD,label=cname7, fmt='-o')
#plt.plot(x,resF,label='detached DC')
plt.plot(x,resF/(1-mProp),label='detached norm DC')
plt.ylabel('Value')
plt.xlabel('Region number')
# show a legend on the plot
plt.legend()
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('expDG_10reg_10dv_3W_density_target_error.png')
#plt.savefig('expDG_10reg_10dv_3W_density_target_error.pdf')
#plt.savefig('expDG_10reg_10dv_3W_density_target_error.svg')

In [ ]:
#plt.plot(x,target,label=cname7)
plt.errorbar(x,target,targSD,label=cname7, fmt='-o')
plt.plot(x,resF,label='detached DC')
#plt.plot(x,resF/(1-mProp),label='detached norm DC')
plt.ylabel('Value')
plt.xlabel('Region number')
# show a legend on the plot
plt.legend()
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('expDG_10reg_10dv_3W_density_target_error.png')
#plt.savefig('expDG_10reg_10dv_3W_density_target_error.pdf')
#plt.savefig('expDG_10reg_10dv_3W_density_target_error.svg')

## Degradation adjusted

In [ ]:
par_dev = par.copy()
dv_val = [-2.372770811153945,-1.2890811921179512,-4.893397765300463,-1.23886467075058,0.9970473115280463,-0.4719839734855036,0.9999889207762749,-5.58412402098368,-5.812258408904807,-6.486561845484665]
par_dev[13:] = [-2.372770811153945,-1.2890811921179512,-4.893397765300463,-1.23886467075058,0.9970473115280463,-0.4719839734855036,0.9999889207762749,-5.58412402098368,-5.812258408904807,-6.486561845484665]
cost = 1.3329819218418034e-14
chi2 = 1.5470576015145376e-08

In [ ]:
model.dinit=dinit
utrace = model.calcUtrace(par_dev)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], model.segIdx, model.expD['Abbreviation'], fun=np.sum)

In [ ]:
{'F' : par_dev[0],
'Ctau' : par_dev[1],
'mProp' : par_dev[2],
'(1-mProp)' : (1-par_dev[2]),
'demand_rd1' : par_dev[3],
'demand_rd2' : par_dev[4],
'demand_rd3' : par_dev[5],
'demand_rd4' : par_dev[6],
'demand_rd5' : par_dev[7],
'demand_rd6' : par_dev[8],
'demand_rd7' : par_dev[9],
'demand_rd8' : par_dev[10],
'demand_rd9' : par_dev[11],
'demand_rd10' : par_dev[12],
#'dv' : par[13],
'dv_rd1' : par_dev[13],
'dv_rd2' : par_dev[14],
'dv_rd3' : par_dev[15],
'dv_rd4' : par_dev[16],
'dv_rd5' : par_dev[17],
'dv_rd6' : par_dev[18],
'dv_rd7' : par_dev[19],
'dv_rd8' : par_dev[20],
'dv_rd9' : par_dev[21],
'dv_rd10' : par_dev[22],
'norm_factor' : (1-mProp),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - target) ** 2),
'err_norm' : np.sum((resF/(1-mProp) - target) ** 2)/tnorm,
'chi2' : np.sum(((resF/(1-mProp) - target)/targSD) ** 2),
'cost' : cost,
'Chi2' : chi2
}

In [ ]:
x=np.array(range(model.expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
ax.plot(x,target,label='Target')
#ax.plot(x,resM,label='mobile DC')
ax.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='norm detached DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('expDG_10reg_10dv_3W_dv_density_target.png')
#plt.savefig('expDG_10reg_10dv_3W_dv_density_target.pdf')
#plt.savefig('expDG_10reg_10dv_3W_dv_density_target.svg')

In [ ]:
#plt.plot(x,target,label=cname7)
plt.errorbar(x,target,targSD,label=cname7, fmt='-o')
#plt.plot(x,resF,label='detached DC')
plt.plot(x,resF/(1-mProp),label='detached norm DC')
plt.ylabel('Value')
plt.xlabel('Region number')
# show a legend on the plot
plt.legend()
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('expDG_10reg_10dv_3W_dv_density_target_error.png')
#plt.savefig('expDG_10reg_10dv_3W_dv_density_target_error.pdf')
#plt.savefig('expDG_10reg_10dv_3W_dv_density_target_error.svg')

In [ ]:
import DG_10reg_10dv_model_dvonly as model3W
model3W.dinit=dinit
utrace = model3W.calcUtrace(dv_val)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], model.segIdx, model.expD['Abbreviation'], fun=np.sum)

In [ ]:
{'F' : par_dev[0],
'Ctau' : par_dev[1],
'mProp' : par_dev[2],
'(1-mProp)' : (1-par_dev[2]),
'demand_rd1' : par_dev[3],
'demand_rd2' : par_dev[4],
'demand_rd3' : par_dev[5],
'demand_rd4' : par_dev[6],
'demand_rd5' : par_dev[7],
'demand_rd6' : par_dev[8],
'demand_rd7' : par_dev[9],
'demand_rd8' : par_dev[10],
'demand_rd9' : par_dev[11],
'demand_rd10' : par_dev[12],
#'dv' : par[13],
'dv_rd1' : par_dev[13],
'dv_rd2' : par_dev[14],
'dv_rd3' : par_dev[15],
'dv_rd4' : par_dev[16],
'dv_rd5' : par_dev[17],
'dv_rd6' : par_dev[18],
'dv_rd7' : par_dev[19],
'dv_rd8' : par_dev[20],
'dv_rd9' : par_dev[21],
'dv_rd10' : par_dev[22],
'norm_factor' : (1-mProp),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - target) ** 2),
'err_norm' : np.sum((resF/(1-mProp) - target) ** 2)/tnorm,
'chi2' : np.sum(((resF/(1-mProp) - target)/targSD) ** 2),
'cost' : cost,
'Chi2' : chi2
}

# 18 month data

In [ ]:
#expD=pd.read_csv('../data/DG_intensity_values_15072024.csv')
expD=pd.read_csv('../data/DG_density_values_15072024.csv')
bgSignal=1e-5
cname0='D0M18'
d0w = -1 * np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d0w[sidx] *= -1*expD[f"{cname0}_MEAN"][i]/len(sidx)
for i in range(N):
    if d0w[i]<0:
        d0w[i] = bgSignal
dinit = d0w/np.sum(d0w)

cname7='D7M18'
d7w = -1*np.ones(N)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d7w[sidx] *= -1 * expD[f"{cname7}_MEAN"][i]/len(sidx)
for i in range(N):
    if d7w[i]<0:
        d7w[i] = bgSignal
target = np.array(expD[f"{cname7}_MEAN"])/np.sum(expD[f"{cname0}_MEAN"]) #norm target to Day0 sum to take into accound degradation
targSD = np.array(expD[f"{cname7}_SD"])/np.sum(expD[f"{cname0}_MEAN"]) #results to fit to
tnorm = np.sum(target ** 2)
day7 = 7 * 24 * 3600 # final time point


In [ ]:
model.dinit=dinit
utrace = model.calcUtrace(par)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], model.segIdx, model.expD['Abbreviation'], fun=np.sum)

In [ ]:
{'F' : par[0],
'Ctau' : par[1],
'mProp' : par[2],
'demand_rd1' : par[3],
'demand_rd2' : par[4],
'demand_rd3' : par[5],
'demand_rd4' : par[6],
'demand_rd5' : par[7],
'demand_rd6' : par[8],
'demand_rd7' : par[9],
'demand_rd8' : par[10],
'demand_rd9' : par[11],
'demand_rd10' : par[12],
'dv' : par[13],
#'dv_rd1' : par[13],
#'dv_rd2' : par[14],
#'dv_rd3' : par[15],
#'dv_rd4' : par[16],
#'dv_rd5' : par[17],
#'dv_rd6' : par[18],
#'dv_rd7' : par[19],
#'dv_rd8' : par[20],
#'dv_rd9' : par[21],
#'dv_rd10' : par[22],
'norm_factor' : (1-mProp),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - target) ** 2),
'err_norm' : np.sum((resF/(1-mProp) - target) ** 2)/tnorm,
'chi2' : np.sum(((resF/(1-mProp) - target)/targSD) ** 2),
'cost' : cost,
'Chi2' : chi2
}

In [ ]:
x=np.array(range(model.expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
ax.plot(x,target,label='Target')
#ax.plot(x,resM,label='mobile DC')
ax.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='norm detached DC')
#ax.plot(x,resF/(1-mProp),label='norm detached DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('expDG_10reg_10dv_18M_density_target.png')
#plt.savefig('expDG_10reg_10dv_18M_density_target.pdf')
#plt.savefig('expDG_10reg_10dv_18M_density_target.svg')

In [ ]:
#plt.plot(x,target,label=cname7)
plt.errorbar(x,target,targSD,label=cname7, fmt='-o')
#plt.plot(x,resF,label='detached DC')
plt.plot(x,resF/(1-mProp),label='detached norm DC')
plt.ylabel('Value')
plt.xlabel('Region number')
# show a legend on the plot
plt.legend()
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('expDG_10reg_10dv_18M_density_target_error.png')
#plt.savefig('expDG_10reg_10dv_18M_density_target_error.pdf')
#plt.savefig('expDG_10reg_10dv_18M_density_target_error.svg')


In [ ]:
iM, iF = sushibelt.aggregate_segments(np.concatenate((list(model.dinit),list(model.dinit))), model.segIdx, model.expD['Abbreviation'], fun=np.sum)
plt.plot(x,iM,label='dinit')
plt.plot(x,expD[f"{cname0}_MEAN"]/np.sum(expD[f"{cname0}_MEAN"]),label=f"{cname0}")
plt.ylabel('Value')
plt.xlabel('Region number')
# show a legend on the plot
plt.legend()
plt.title("dinit") 
# function to show the plot
plt.show()


## Degradation adjusted

In [ ]:
par_dev = par.copy()
#par_dev[13:] = [
cost = 1.3329819218418034e-14
chi2 = 1.5470576015145376e-08

In [ ]:
model.dinit=dinit
utrace = model.calcUtrace(par_dev)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], model.segIdx, model.expD['Abbreviation'], fun=np.sum)

In [ ]:
{'F' : par_dev[0],
'Ctau' : par_dev[1],
'mProp' : par_dev[2],
'(1-mProp)' : (1-par_dev[2]),
'demand_rd1' : par_dev[3],
'demand_rd2' : par_dev[4],
'demand_rd3' : par_dev[5],
'demand_rd4' : par_dev[6],
'demand_rd5' : par_dev[7],
'demand_rd6' : par_dev[8],
'demand_rd7' : par_dev[9],
'demand_rd8' : par_dev[10],
'demand_rd9' : par_dev[11],
'demand_rd10' : par_dev[12],
#'dv' : par[13],
'dv_rd1' : par_dev[13],
'dv_rd2' : par_dev[14],
'dv_rd3' : par_dev[15],
'dv_rd4' : par_dev[16],
'dv_rd5' : par_dev[17],
'dv_rd6' : par_dev[18],
'dv_rd7' : par_dev[19],
'dv_rd8' : par_dev[20],
'dv_rd9' : par_dev[21],
'dv_rd10' : par_dev[22],
'norm_factor' : (1-mProp),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - model.target) ** 2),
'err_norm' : np.sum((resF/(1-mProp) - model.target) ** 2)/tnorm,
'chi2' : np.sum(((resF/(1-mProp) - model.target)/model.targSD) ** 2),
'cost' : cost,
'Chi2' : chi2
}

In [ ]:
x=np.array(range(model.expD.shape[0]),dtype=int)
fig, ax = plt.subplots()
ax.plot(x,target,label='Target')
#ax.plot(x,resM,label='mobile DC')
ax.plot(x,resF,label='detached DC')
ax.plot(x,resF/(1-mProp),label='norm detached DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
#plt.show()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('expDG_10reg_10dv_3W_dv_density_target.png')
#plt.savefig('expDG_10reg_10dv_3W_dv_density_target.pdf')
#plt.savefig('expDG_10reg_10dv_3W_dv_density_target.svg')

In [ ]:
#plt.plot(x,target,label=cname7)
plt.errorbar(x,target,targSD,label=cname7, fmt='-o')
#plt.plot(x,resF,label='detached DC')
plt.plot(x,resF/(1-mProp),label='detached norm DC')
plt.ylabel('Value')
plt.xlabel('Region number')
# show a legend on the plot
plt.legend()
plt.title(f"Cost = {format(np.sum((resF/(1-mProp) - target) ** 2),'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - target)/targSD) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('expDG_10reg_10dv_3W_dv_density_target_error.png')
#plt.savefig('expDG_10reg_10dv_3W_dv_density_target_error.pdf')
#plt.savefig('expDG_10reg_10dv_3W_dv_density_target_error.svg')